# 1) Titolo e obiettivi
Lezione 36: Knowledge Mining con TF-IDF e ricerca semantica semplice.

---

## Mappa concettuale della lezione

```
KNOWLEDGE MINING - DAL TESTO ALLA CONOSCENZA
=============================================

LIVELLO 1: RECUPERO              LIVELLO 2: ESTRAZIONE
+----------------------+         +----------------------+
|  Information         |         |  Knowledge           |
|  Retrieval           |   -->   |  Extraction          |
|  "trova documenti"   |         |  "estrai fatti"      |
+----------------------+         +----------------------+
         |                                |
         v                                v
+----------------------+         +----------------------+
| Query -> Ranking     |         | Entita, Relazioni    |
| TF-IDF, coseno       |         | Pattern, Regole      |
+----------------------+         +----------------------+


PIPELINE KNOWLEDGE MINING
=========================

    +------------------+
    |  CORPUS GREZZO   |
    |  (testi, PDF...) |
    +--------+---------+
             |
             v
    +------------------+
    | PREPROCESSING    |
    | clean, tokenize  |
    +--------+---------+
             |
    +--------+---------+
    |                  |
    v                  v
+----------+    +-------------+
| RICERCA  |    | ESTRAZIONE  |
| TF-IDF   |    | NER, Regex  |
| ranking  |    | patterns    |
+----+-----+    +------+------+
     |                 |
     v                 v
+----------+    +-------------+
| DOCUMENTI|    | FATTI       |
| RILEVANTI|    | STRUTTURATI |
+----+-----+    +------+------+
     |                 |
     +--------+--------+
              |
              v
    +------------------+
    |  KNOWLEDGE BASE  |
    |  interrogabile   |
    +------------------+
```

---

## Obiettivi didattici

| # | Obiettivo | Livello |
|---|-----------|---------|
| 1 | Distinguere IR da Knowledge Mining | Concettuale |
| 2 | Implementare ricerca TF-IDF su corpus custom | Operativo |
| 3 | Combinare ricerca e estrazione in pipeline | Integrazione |
| 4 | Gestire aggiornamenti incrementali del corpus | Avanzato |
| 5 | Valutare limiti del matching lessicale | Critico |
| 6 | Preparare transizione verso embeddings | Prospettiva |

---

## Concetti chiave

> **Knowledge Mining**: processo di estrazione di conoscenza strutturata (fatti, relazioni, pattern) da dati non strutturati (testo, documenti).

> **Differenza IR vs KM**: IR trova documenti rilevanti; KM estrae informazioni specifiche dai documenti trovati.

> **Gap semantico**: TF-IDF cattura solo match lessicali; "auto" e "macchina" non matchano anche se sinonimi.

---

## Limiti del matching lessicale

```
QUERY: "problemi cardiaci"

DOCUMENTO A: "Il paziente presenta aritmia e insufficienza cardiaca"
             ^^^^^^^^ match parziale ("cardiaca" ~ "cardiaci")

DOCUMENTO B: "Problemi al cuore riscontrati durante l'esame"
             ^^^^^^^^ match ("problemi") ma "cuore" =/= "cardiaci"

DOCUMENTO C: "Patologie cardiovascolari in aumento"
             NO MATCH LESSICALE (sinonimi, non termini identici)

TF-IDF RANKING: A > B > C
RILEVANZA REALE: A = B = C (tutti rilevanti!)
```

**Soluzione**: sinonimi manuali, stemming, o passaggio a embeddings semantici.

---

## Cosa useremo
- `TfidfVectorizer` per rappresentazione vettoriale
- `cosine_similarity` per ranking
- Pattern regex per estrazione base
- Dizionari di sinonimi per espansione

## Prerequisiti
- Information Retrieval base (Lezione 35)
- TF-IDF e similarita' (Lezione 31)
- Regex e pattern matching (Lezione 34)


# 2) Teoria concettuale approfondita
- Knowledge mining: trasformare documenti in vettori per recuperarli con query.
- TF-IDF: pesa termini in base a frequenza/rarita'; utile per ranking.
- Similarita' coseno: misura di vicinanza tra query e documenti.


# 3) Schema mentale / mappa decisionale
1. Raccogli documenti e pulisci testo.
2. Fit del TF-IDF sul corpus; conserva vocabolario.
3. Trasforma le query con lo stesso vectorizer.
4. Calcola similarita' coseno e ordina i documenti.
5. Valida risultati e aggiusta stopword/sinonimi.


# 4) Sezione dimostrativa
Demo: pipeline TF-IDF, funzione di ricerca, due query di esempio.


In [ ]:
# Setup librerie
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')
np.random.seed(42)


In [ ]:
# Corpus simulato
documents = [
    "Politica estera e nuove alleanze in Europa",
    "Analisi finanziaria e bilancio trimestrale dell'azienda",
    "Guida pratica alla richiesta di rimborsi per viaggi",
    "Manuale operativo per assistenza clienti e ticketing",
    "Tutorial di machine learning applicato al marketing",
]
print(f"Documenti: {len(documents)}")
assert len(documents) > 0


In [ ]:
# Vettorizzazione TF-IDF con stopword italiane
vectorizer = TfidfVectorizer(stop_words='italian')
X = vectorizer.fit_transform(documents)
print(f"Shape TF-IDF: {X.shape}, sparsimita': {1 - X.nnz/(X.shape[0]*X.shape[1]):.1%}")
assert X.shape[0] == len(documents)


In [ ]:
# Funzione di ricerca con similarita' coseno

def search(query, top_k=3):
    q_vec = vectorizer.transform([query])
    sims = cosine_similarity(q_vec, X).flatten()
    order = sims.argsort()[::-1][:top_k]
    return [(i, sims[i], documents[i]) for i in order]


In [ ]:
# Query di esempio
queries = ["richiesta rimborsi viaggio", "marketing machine learning"]
for q in queries:
    print(f"
Query: {q}")
    for idx, score, doc in search(q):
        print(f"  Score {score:.3f} -> {doc}")


### Osservazioni
- Le query recuperano documenti con overlap lessicale; sinonimi non gestiti.
- Sparsimita' alta: controlla stopword e vocabolario per non degradare le prestazioni.


# 5) Esercizi svolti (step-by-step)
## Esercizio 36.1 - Aggiungere un documento e rilanciare query


In [ ]:
# Esercizio 36.1
new_doc = "Procedura dettagliata per rimborsi di biglietti aerei e hotel"
corpus2 = documents + [new_doc]
X2 = vectorizer.fit_transform(corpus2)
q_vec = vectorizer.transform([queries[0]])
sims = cosine_similarity(q_vec, X2).flatten()
order = sims.argsort()[::-1][:3]
print("Top3 con nuovo documento:")
for i in order:
    print(i, sims[i], corpus2[i])


## Esercizio 36.2 - Stopword personalizzate


In [ ]:
# Esercizio 36.2
custom_stop = ['analisi','guida','manuale']
vec_custom = TfidfVectorizer(stop_words=set(custom_stop + list(TfidfVectorizer(stop_words='italian').get_stop_words())))
Xc = vec_custom.fit_transform(documents)
print(f"Sparsimita' con stopword custom: {1 - Xc.nnz/(Xc.shape[0]*Xc.shape[1]):.1%}")


## Esercizio 36.3 - Sinonimi manuali


In [ ]:
# Esercizio 36.3
synonyms = {'rimborsi': 'rimborso', 'refund': 'rimborso'}
normalized = []
for doc in documents:
    tmp = doc
    for k,v in synonyms.items():
        tmp = tmp.replace(k, v)
    normalized.append(tmp)

vec_norm = TfidfVectorizer(stop_words='italian')
Xn = vec_norm.fit_transform(normalized)
print("Vocabolario normalizzato (prime 10 parole):", list(vec_norm.get_feature_names_out())[:10])


# 6) Conclusione operativa - Bignami Knowledge Mining

---

## I 5 Take-Home Messages

| # | Concetto | Perche' conta |
|---|----------|---------------|
| 1 | **KM = IR + Estrazione** | Non basta trovare documenti, devi estrarre fatti |
| 2 | **TF-IDF e' baseline, non soluzione finale** | Match lessicale ha gap semantico intrinseco |
| 3 | **Sinonimi manuali colmano gap parzialmente** | Dizionari di dominio migliorano recall |
| 4 | **Vocabolario fisso richiede aggiornamento** | Corpus che cresce -> refit periodico |
| 5 | **Embeddings superano limiti lessicali** | Prossimo passo per semantic search vera |

---

## IR vs Knowledge Mining

```
+------------------+------------------+------------------+
|   ASPETTO        |   IR             |   KNOWLEDGE      |
|                  |                  |   MINING         |
+------------------+------------------+------------------+
| Output           | Lista documenti  | Fatti strutturati|
| Granularita'     | Documento intero | Entita/relazioni |
| Query            | Testo libero     | Pattern specifici|
| Valutazione      | Precision/Recall | Accuracy estratto|
| Uso tipico       | Motori di ricerca| Knowledge graphs |
+------------------+------------------+------------------+
```

---

## Pipeline combinata IR + Estrazione

```
QUERY UTENTE                    PATTERN ESTRAZIONE
     |                                |
     v                                v
+---------+                    +-----------+
| TF-IDF  |                    | Regex/NER |
| search  |                    | rules     |
+----+----+                    +-----+-----+
     |                               |
     v                               v
+------------------+          +------------------+
| TOP-K DOCUMENTI  |   --->   | ESTRAI FATTI DA  |
| rilevanti        |          | CIASCUN DOC      |
+------------------+          +------------------+
                                     |
                                     v
                              +------------------+
                              | AGGREGAZIONE     |
                              | dedup, merge     |
                              +------------------+
                                     |
                                     v
                              +------------------+
                              | KNOWLEDGE BASE   |
                              | strutturata      |
                              +------------------+
```

---

## Template operativo

```python
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re

class KnowledgeMiner:
    def __init__(self, corpus, synonyms=None):
        self.corpus = corpus
        self.synonyms = synonyms or {}
        self.vectorizer = TfidfVectorizer(stop_words='english')
        self.doc_matrix = self.vectorizer.fit_transform(corpus)
    
    def search(self, query, top_k=5):
        # Espandi query con sinonimi
        expanded = self._expand(query)
        q_vec = self.vectorizer.transform([expanded])
        scores = cosine_similarity(q_vec, self.doc_matrix).flatten()
        top_idx = scores.argsort()[::-1][:top_k]
        return [(i, self.corpus[i], scores[i]) for i in top_idx]
    
    def _expand(self, query):
        tokens = query.lower().split()
        expanded = tokens.copy()
        for t in tokens:
            expanded.extend(self.synonyms.get(t, []))
        return ' '.join(expanded)
    
    def extract_from_results(self, results, pattern):
        """Estrae fatti dai documenti trovati"""
        facts = []
        for idx, doc, score in results:
            matches = re.findall(pattern, doc, re.IGNORECASE)
            facts.extend([(idx, m, score) for m in matches])
        return facts
    
    def add_documents(self, new_docs):
        """Aggiunge documenti e reindicizza"""
        self.corpus.extend(new_docs)
        self.doc_matrix = self.vectorizer.fit_transform(self.corpus)

# Uso
miner = KnowledgeMiner(corpus, synonyms={'auto': ['macchina', 'veicolo']})
results = miner.search("problemi auto")
facts = miner.extract_from_results(results, r'\b[A-Z]{2}\d{3}[A-Z]{2}\b')  # targhe
```

---

## Errori comuni e soluzioni

| Errore | Conseguenza | Soluzione |
|--------|-------------|-----------|
| Ignorare gap semantico | Bassa recall su sinonimi | Dizionari, stemming, embeddings |
| Non aggiornare indice | Nuovi doc non trovabili | Refit periodico o incrementale |
| Estrazione senza validazione | Falsi positivi | Pattern precisi + post-filtering |
| Corpus troppo piccolo | Vocabolario povero | Aumentare dati o usare pretrained |

---

## Metodi e concetti chiave

| Elemento | Ruolo |
|----------|-------|
| `TfidfVectorizer` | Indicizzazione corpus |
| `cosine_similarity` | Ranking per rilevanza |
| Dizionario sinonimi | Espansione query manuale |
| Regex patterns | Estrazione fatti |
| Refit | Aggiornamento indice |


# 7) Checklist di fine lezione
- [ ] Ho pulito il corpus e scelto stopword adeguate.
- [ ] Ho fit il TF-IDF sul corpus e usato transform sulle query.
- [ ] Ho controllato sparsimita' e dimensione del vocabolario.
- [ ] Ho testato almeno due query e valutato i risultati.
- [ ] Ho documentato normalizzazioni/sinonimi applicati.

Glossario
- TF-IDF: peso termini per frequenza/rarita'.
- Similarita' coseno: misura di vicinanza tra vettori.
- Stopword: termini poco informativi rimossi dal vocabolario.
- Vocabolario: elenco di termini indicizzati dal vectorizer.


# 8) Changelog didattico

| Versione | Data | Modifiche |
|----------|------|-----------|
| 1.0 | 2024-01-XX | Struttura iniziale 8 sezioni |
| 2.0 | 2024-12-XX | Espansione completa Knowledge Mining |
| 2.1 | - | Pipeline ASCII a due livelli (IR + Estrazione) |
| 2.2 | - | Esempio gap semantico con documenti medici |
| 2.3 | - | Confronto tabellare IR vs Knowledge Mining |
| 2.4 | - | Classe KnowledgeMiner completa con metodi |
| 2.5 | - | Gestione aggiornamento incrementale corpus |
| 2.6 | - | Transizione concettuale verso embeddings |

---

## Note di versione

**v2.0 - Espansione didattica completa**
- Posizionamento KM come evoluzione di IR
- Visualizzazione esplicita del gap semantico
- Pipeline combinata ricerca + estrazione
- Classe template riutilizzabile per progetti
- Emphasis su limiti TF-IDF e prossimi passi
- Preparazione per embeddings e semantic search

**Dipendenze didattiche**
- Richiede: Lezione 35 (IR base), Lezione 34 (patterns)
- Prepara: Lezione 37+ (Deep Learning, embeddings)
